In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral")

In [2]:
llm.invoke("tell me a short quote from someone great")

' "Be the change you wish to see in the world." - Mahatma Gandhi'

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_path = "https://www.langchain.com/langsmith"
)

docs = loader.load()

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True
)


In [5]:
all_splits = text_splitter.split_documents(docs)

In [11]:
from langchain_community import embeddings

embedding = embeddings.OllamaEmbeddings(
    model = "nomic-embed-text"
)

In [12]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding
)

In [13]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = """Given a Chat History and the latest user question \ 
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question \ 
just reformulate it if needed and otherwise return it as it is."""

In [17]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [18]:
from langchain_core.output_parsers import StrOutputParser

contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [19]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_chain.invoke(
    {
        "chat_history":[
            HumanMessage(content = "What Does LLM Stand For?"),
            AIMessage(content="Large language model"),
        ],
        "question" : "What ius meant by large ?"
    }
)

' What do you mean by "large" in the context of a large language model?'